In [1]:
import spotipy as spot
import pandas as pd
import json

In [2]:
config = json.load(open(file="config.json", encoding="utf-8"))
config

{'CLIENT_ID': '468b8b024bfb41d5b1957dad2afc766a',
 'CLIENT_SECRET': '8827668f8ed64f13bf8c2e83781c3997',
 'REDIRECT_URI': 'http://localhost:8080'}

In [3]:
# Authorization Flow that WORKS
try:
    token = spot.util.prompt_for_user_token(username='frostcow',
        scope=["user-read-recently-played",
            "playlist-modify-public",
            "user-library-read",],
        client_id=config["CLIENT_ID"],
        client_secret=config["CLIENT_SECRET"],
        redirect_uri=config["REDIRECT_URI"])
    test_sp = spot.Spotify(auth=token)
except spot.SpotifyOauthError:
    print("User declined")

In [4]:
type(token)

str

# General purpose testing grounds for function alterations or new code below

In [5]:
from main import get_last_50_songs, get_saved_tracks

last = get_last_50_songs(test_sp)
saved = get_saved_tracks(test_sp)

In [6]:
saved.head()
last.head()

,Track ID,Song Name,Album Name,Artist Name,Explicit,Duration,Song Popularity,Album Release Year
0,6X4fTprmm1elJ6LPGXJEsP,Santolea,Santolea,Xaverius Love,False,151250,58,2022
1,5LIsR2QFLqxot9tX9huYwi,From Above,Your Story,Albin Svensson,False,106013,64,2022
2,79v9cKSakZbCRZI0r0QrV7,The Last Leaf Falls,The Last Leaf Falls,Josh Kramer,False,125667,51,2022
3,2u6LWpFrDoipclUF01Oaeu,Life Goes On - Hideyuki Hashimoto Rework,Life Goes On (Hideyuki Hashimoto Rework),Kevin Imbrechts,False,188506,58,2022
4,3yMBkcCZA5dTrqEoz83azz,Evenfall,Evenfall,Stephen Weber,False,181786,59,2022


In [7]:
from content_based import ohe

o_saved, o_last = ohe(saved=saved, last=last)

In [8]:
o_last.head()

,Track ID,Song Name,Album Name,Artist Name,Duration,Song Popularity,Album Release Year,0
0,6X4fTprmm1elJ6LPGXJEsP,Santolea,Santolea,Xaverius Love,151250,58,2022,1.0
1,5LIsR2QFLqxot9tX9huYwi,From Above,Your Story,Albin Svensson,106013,64,2022,1.0
2,79v9cKSakZbCRZI0r0QrV7,The Last Leaf Falls,The Last Leaf Falls,Josh Kramer,125667,51,2022,1.0
3,2u6LWpFrDoipclUF01Oaeu,Life Goes On - Hideyuki Hashimoto Rework,Life Goes On (Hideyuki Hashimoto Rework),Kevin Imbrechts,188506,58,2022,1.0
4,3yMBkcCZA5dTrqEoz83azz,Evenfall,Evenfall,Stephen Weber,181786,59,2022,1.0


In [11]:
from content_based import encode_year

e_saved, e_last = encode_year(saved=saved, last=last)

In [12]:
e_last.head()

,Track ID,Song Name,Album Name,Artist Name,Explicit,Duration,Song Popularity,Album Release Year,Year
0,6X4fTprmm1elJ6LPGXJEsP,Santolea,Santolea,Xaverius Love,False,151250,58,2022,25
1,5LIsR2QFLqxot9tX9huYwi,From Above,Your Story,Albin Svensson,False,106013,64,2022,25
2,79v9cKSakZbCRZI0r0QrV7,The Last Leaf Falls,The Last Leaf Falls,Josh Kramer,False,125667,51,2022,25
3,2u6LWpFrDoipclUF01Oaeu,Life Goes On - Hideyuki Hashimoto Rework,Life Goes On (Hideyuki Hashimoto Rework),Kevin Imbrechts,False,188506,58,2022,25
4,3yMBkcCZA5dTrqEoz83azz,Evenfall,Evenfall,Stephen Weber,False,181786,59,2022,25
